In [8]:
import numpy as np
import pandas as pd
import json
import re

In [9]:
# load json as dict
with open('/Users/khnhkd/WorkSpace/python_proj/real-estate-trading-/crawl-data-bds/data/data_thread_1.json') as json_file:
    dict_1 = json.load(json_file)

with open('/Users/khnhkd/WorkSpace/python_proj/real-estate-trading-/crawl-data-bds/data/data_thread_2.json') as json_file:
    dict_2 = json.load(json_file)

with open('/Users/khnhkd/WorkSpace/python_proj/real-estate-trading-/crawl-data-bds/data/data_thread_3.json') as json_file:
    dict_3 = json.load(json_file)

with open('/Users/khnhkd/WorkSpace/python_proj/real-estate-trading-/crawl-data-bds/data/data_thread_4.json') as json_file:
    dict_4 = json.load(json_file)

all_records = dict_1 + dict_2 + dict_3 + dict_4

# delete some key
for i in range(len(all_records)):
    del all_records[i]['url']

In [10]:
def split_number(data):
    try:
        return re.findall('[(\d+(\.\d+)?)|(\.?\d+)]+', data)[0]
    except:
        return None

def extract_million(data):
    if 'triệu' in data:
        try:
            return float(re.findall('[(\d+(\.\d+)?)|(\.?\d+)]+', data)[0])
        except:
            return 0
    return 0

def extract_billion(data):
    if 'tỷ' in data:
        try:
            return float(re.findall('[(\d+(\.\d+)?)|(\.?\d+)]+', data)[0])*1000
        except:
            return 0
    return 0

def extract_price(df):
    df['million'] = df['price'].apply(extract_million).astype(np.float64)
    df['billion'] = df['price'].apply(extract_billion).astype(np.float64)
    df['billion_to_million'] = df['billion'] / df['acreage']
    
    df['price'] = df['billion_to_million'] + df['million']
    df['price'] = df['price'].replace(0, np.nan)
    
    return df.drop(['million', 'billion', 'billion_to_million'], axis=1)

    
def process_data(df):
    # extract bedroom, toilet, acreage to number values
    df['bedroom'] = df['bedroom'].apply(split_number).astype(np.float64)
    df['toilet'] = df['toilet'].apply(split_number).astype(np.float64)
    df['acreage'] = df['acreage'].apply(split_number).astype(np.float64)

    # extract address to district values
    
    # drop nan value of columns address, acreage, bedroom
#     df = df.dropna(subset=['address', 'acreage', 'bedroom'])
    
    # split price million and billion
    df = extract_price(df)
    
    return df

In [11]:
df = pd.DataFrame(all_records)
df = process_data(df).replace({np.nan: None})
df

,title,address,bedroom,toilet,investor,acreage,price,lat,long,distance,images,description
0,em vinh có quỹ 2pn - 3pn tầng đẹp và giá tốt ...,"dự án the matrix one, đường lê quang đạo, phườ...",2.0,2.0,công ty cổ phần đầu tư mai linh,88.0,None,21.009386,105.773872,8.453731,[https://file4.batdongsan.com.vn/resize/200x20...,em vinh: 0967.619.366.1. căn hộ siêu cấp hạng ...
1,chính chủ bán căn hộ chung cư starcity mặt đư...,"dự án starcity lê văn lương, đường lê văn lươn...",3.0,2.0,ocean group - tập đoàn đại dương,111.7,32.2,21.006182,105.806427,5.413824,[https://file4.batdongsan.com.vn/resize/200x20...,chính chủ bán căn hộ chung cư starcity mặt đườ...
2,"quỹ căn đẹp giá tốt, ck cực cao, htls 0% đến ...","dự án vinhomes ocean park gia lâm, xã đa tốn, ...",2.0,2.0,tập đoàn vingroup,68.4,27.7,20.996325,105.932869,9.104289,[https://file4.batdongsan.com.vn/resize/200x20...,"trực tiếp cđt, chính sách hỗ trợ tốt nhất năm,..."
3,chính chủ bán căn hộ chung cư giá hời! full n...,"phố lạc trung, phường vĩnh tuy, hai bà trưng,...",2.0,2.0,None,65.0,29.2,21.002986,105.865675,3.191921,[https://file4.batdongsan.com.vn/resize/200x20...,trung tâm phố ngân hàng - văn phòng tại con ph...
4,trực tiếp cđt ch the matrix one mỹ đình ck 11...,"dự án the matrix one, đường lê quang đạo, phườ...",2.0,2.0,công ty cổ phần đầu tư mai linh,88.0,None,21.009386,105.773872,8.453731,[https://file4.batdongsan.com.vn/resize/200x20...,mr tuấn phụ trách dự án: 0911228183.1. chính s...
...,...,...,...,...,...,...,...,...,...,...,...,...
16395,cần tiền gấp bán lỗ căn 1206 - dv03 rose town...,"rose town, 79, đường ngọc hồi, phường hoàng li...",2.0,2.0,None,71.0,27.5,20.956589,105.844887,8.087054,[https://file4.batdongsan.com.vn/resize/200x20...,gia đình chú tôi có 1 suất ngoại giao trong di...
16396,"chính chủ cần bán gấp căn 2 phòng ngủ, 78m2, ...","dự án park hill premium - times city, đường mi...",2.0,2.0,tập đoàn vingroup,78.0,44.9,20.996578,105.865196,3.83108,[https://file4.batdongsan.com.vn/resize/200x20...,do nhu cầu chuyển nhà nên gia đình cần bán nha...
16397,"sở hữu căn 2 pn chỉ từ 1,7 tỷ tại le grand ja...","dự án le grand jardin sài đồng, đường huỳnh vă...",2.0,2.0,công ty cp đầu tư xây dựng số 9 hà nội,54.0,None,21.035978,105.906296,5.644863,[https://file4.batdongsan.com.vn/resize/200x20...,các hạng mục công trình dự án le grand jardin ...
16398,cần bán chung cư tầng 1 tại phường giang biên...,"dự án việt hưng, phường giang biên, long biên,...",2.0,None,tổng công ty đầu tư phát triển nhà và đô thị hud,85.0,30.6,21.062691,105.908333,6.914078,[],"cần bán chung cư tầng 1 tại phường giang biên,..."


In [12]:
js = df.to_dict('records')

In [13]:
len(js)

16400

In [14]:
with open('data-to-db.json', 'w', encoding='utf-8') as file:
    json.dump(js, file, ensure_ascii=False)